# Prepare well data and reservoir horizon

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import time
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import pylab as py
from mpl_toolkits import mplot3d
import math
from matplotlib import rcParams

# Create directories to store data
!mkdir -p ../dat

import seppy

rcParams['font.size'] = 12
rcParams['font.family'] = 'sans-serif'
rcParams['text.usetex'] =True

### Utility functions

In [ ]:
def getXYZ(md, traj,md0=-1):
    """Compute x,y,z coordinates for a given measured depth"""

    n = traj.shape[1]
    
    if md0<0:
        l=traj[2,0]
    else:
        l=md0
    i=1
    while ((l<md) and (i<n)):
        l += math.sqrt((traj[2,i]-traj[2,i-1])**2+(traj[1,i]-traj[1,i-1])**2+(traj[0,i]-traj[0,i-1])**2)
        i += 1
        
    if (i==n):
        print("Warning: the end of the trajectory has been reached")
        x=traj[0,n-1]
        y=traj[1,n-1]
        z=traj[2,n-1]
    else:
        imin = i - 2
        imax=imin+1
        if (imin<0):
            imin=0
            imax=0
        lmin=l-math.sqrt((traj[2,imax]-traj[2,imax-1])**2+(traj[1,imax]-traj[1,imax-1])**2+(traj[0,imax]-traj[0,imax-1])**2)
        wmax=(md-lmin)/(l-lmin)
        wmin=1-wmax
        x=wmin*traj[0,imin]+wmax*traj[0,imax]
        y=wmin*traj[1,imin]+wmax*traj[1,imax]
        z=wmin*traj[2,imin]+wmax*traj[2,imax]
    
    return x,y,z

def ChannelToDepth(Channel):
    """Compute DAS channel measured depth from channel number (between 1 and 3776)
    The conversion function is provided by Chevron"""
    
    FibreLengthMultiplier = 1.02095238387
    ZeroOffset_m          =-2.41333217886
    SpatialResolution_m   = 1.0
    DasFibreStretch       = 1.0056

    Depth_in_meters       = ( (float(Channel) 
                             * SpatialResolution_m 
                             * FibreLengthMultiplier )
                             + ZeroOffset_m) / DasFibreStretch

    return Depth_in_meters

def rotationY(angle):
    """Build rotation matrix around y-axis ; the angle is in degrees"""

    c = math.cos(math.radians(angle))
    s = math.sin(math.radians(angle))
    R=np.array(([c,0,-s],[0,1,0],[s,0,c]))

    return R

def rotationZ(angle):
    """Build rotation matrix around z-axis ; the angle is in degrees"""

    c = math.cos(math.radians(angle))
    s = math.sin(math.radians(angle))
    R=np.array(([c,-s,0],[s,c,0],[0,0,1]))

    return R

def rotateYZ(xyz,theta,phi):
    """Rotate coordinates around y- and z-axes ; angles are in degrees"""

    Ry = rotationY(theta)
    Rz = rotationZ(phi)
    v = np.matmul(Rz,xyz)
    v = np.matmul(Ry,v)

    return v

### Offset vertical well logs

The file contains 6 columns: Measured Depth (ft), P slowness (us/ft), S-fast slowness (us/ft), S-slow slowness (us/ft), Bulk density modeled (g/cc), Normalized Gamma ray (unitless)

In [ ]:
inputfile="../../../input_data/auxiliary_data/OffsetVerticalWell_WellLog.csv"

df=pd.read_csv(inputfile, sep=',',header=1)
data=df.values[:,0:6]

# assume that the MD is the TVD, then convert to m
md_vert = 0.3048*data[:,0]

# convert slownesses to velocity (m/s)
vp_vert = 0.3048*1e+06/data[:,1]
vs_fast_vert = 0.3048*1e+06/data[:,2]
vs_slow_vert = 0.3048*1e+06/data[:,3] # same as vs_slow_vert

# convert density to kg/m3
rho_vert = 1000*data[:,4]

gamma_vert = data[:,5]

# replace the first few negative values of Vs and gamma with the first positive value
vs_slow_vert[:40]=vs_slow_vert[40]
vs_fast_vert[:40]=vs_fast_vert[40]
gamma_vert[:40]=gamma_vert[40]

In [ ]:
# QC plot
plt.figure(figsize=(16,4))
plt.subplot(1,4,1)
plt.plot(vp_vert,md_vert)
plt.gca().invert_yaxis()
plt.ylabel('MD (m)')
plt.xlabel('Velocity (m/s)')
plt.title('Offset well Vp0')
plt.hlines(1930,xmin=2000,xmax=6000,colors='k',linestyles='dashed',linewidth=1)
plt.hlines(1945,xmin=2000,xmax=6000,colors='k',linestyles='dashed',linewidth=1)

plt.subplot(1,4,2)
plt.plot(vs_fast_vert,md_vert)
plt.gca().invert_yaxis()
plt.ylabel('MD (m)')
plt.xlabel('Velocity (m/s)')
plt.title('Offset well Vs0')

plt.subplot(1,4,3)
plt.plot(rho_vert,md_vert)
plt.gca().invert_yaxis()
plt.ylabel('MD (m)')
plt.xlabel('Density (kg/m3)')
plt.title('Offset well density')

plt.subplot(1,4,4)
plt.plot(gamma_vert,md_vert)
plt.gca().invert_yaxis()
plt.ylabel('MD (m)')
plt.xlabel('Gamma ray (unitless)')
plt.title('Offset well normalized gamma ray')

plt.tight_layout()

d1=0.1524 # vertical sampling in meters (=0.5 ft)
print("n=%d o=%f d=%f\n" %(md_vert.shape[0],md_vert[0],d1))

In [ ]:
# save to disk and convert to seplib
outputpath='../dat/'

np.save(outputpath+'ch0_offset_vp',vp_vert)
np.save(outputpath+'ch0_offset_vs_slow',vs_slow_vert)
np.save(outputpath+'ch0_offset_rho',rho_vert)
np.save(outputpath+'ch0_offset_gamma',gamma_vert)

!sep2numpy.py --input ../dat/ch0_offset_vp.npy --output ../dat/ch0_offset_vp.H --mode 1 --datapath ${PWD}/../dat/
!sep2numpy.py --input ../dat/ch0_offset_vs_slow.npy --output ../dat/ch0_offset_vs_slow.H --mode 1 --datapath ${PWD}/../dat/
!sep2numpy.py --input ../dat/ch0_offset_rho.npy --output ../dat/ch0_offset_rho.H --mode 1 --datapath ${PWD}/../dat/
!sep2numpy.py --input ../dat/ch0_offset_gamma.npy --output ../dat/ch0_offset_gamma.H --mode 1 --datapath ${PWD}/../dat/

!echo n1=2085 o1=1691.64 d1=0.1524 label1="MD (m)" >> ../dat/ch0_offset_vp.H
!echo n1=2085 o1=1691.64 d1=0.1524 label1="MD (m)" >> ../dat/ch0_offset_vs_slow.H
!echo n1=2085 o1=1691.64 d1=0.1524 label1="MD (m)" >> ../dat/ch0_offset_rho.H
!echo n1=2085 o1=1691.64 d1=0.1524 label1="MD (m)" >> ../dat/ch0_offset_gamma.H

!rm -f ../dat/ch0_offset_vp.npy ../dat/ch0_offset_vs_slow.npy ../dat/ch0_offset_rho.npy ../dat/ch0_offset_gamma.npy

### Formation bottom horizon

In [ ]:
# Read the deviated well (monitor) trajectory
# The file contains 4 columns: Measured Depth (ft), Inclination (degrees), NS (ft), EW (ft)

inputfile="../../../input_data/auxiliary_data/MonitorWell_WellSurvey.csv"

df=pd.read_csv(inputfile, sep=',',header=0)
data=df.values[:,0:4]

# convert the MD to TVD and convert (ft) to (m)
n=data.shape[0]
z = np.zeros(n)

x = data[:,3]*0.3048
y = data[:,2]*0.3048

z[0] = 0
for i in range(1, n):
    z[i] = z[i-1] + (data[i,0]-data[i-1,0])*math.cos(math.radians(data[i,1]))

z = z*0.3048

# save the trajectory
trajectory = np.array([x,y,z])

# conversion table between MD and TVD
tvd = np.zeros((n, 2))
tvd[:,0] = 0.3048*data[:,0]
tvd[:,1] = z



# Read the formation bottom horizon
inputfile="../../../input_data/auxiliary_data/MonitorWell_Formation_Bottom.csv"

df=pd.read_csv(inputfile, sep=',',header=0)
data=df.values

# apply bulk shift of 48 ft to match the monitor well trajectory and convert (ft) to (m)
n_hrz=data.shape[0]
tvd_hrz = np.zeros((n_hrz, 2))
tvd_hrz[:,0] = 0.3048*data[:,0]
tvd_hrz[:,1] = 0.3048*(data[:,1]-48)


# compute x,y coordinates from TVD
x_hrz = np.zeros(n_hrz)
y_hrz = np.zeros(n_hrz)
z_hrz = np.zeros(n_hrz)

for j in range(n_hrz):
    i=0
    while (tvd_hrz[j,0]>tvd[i,0]):
        i+=1
    a=(tvd_hrz[j,0]-tvd[i-1,0])/(tvd[i,0]-tvd[i-1,0])
    x_hrz[j] = a*x[i] + (1-a)*x[i-1]
    y_hrz[j] = a*y[i] + (1-a)*y[i-1]
    z_hrz[j] = tvd_hrz[j,1]

# compute x,y coordinates using trajectory for verification
formation=np.zeros((n_hrz,3))
for j in range(n_hrz):
    formation[j,:]=getXYZ(tvd_hrz[j,0], trajectory)
    formation[j,2] = z_hrz[j]

In [ ]:
# QC plot
ax = plt.axes(projection='3d')
ax.plot3D(x_hrz,y_hrz,z_hrz,linewidth=3,color='k',label="curve 1")
ax.plot3D(formation[:,0],formation[:,1],formation[:,2],linewidth=1,color='r',label="curve 2")
plt.legend()
plt.gca().invert_zaxis()
plt.gca().set_zlim(1980,1940)
ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")
ax.set_zlabel("Z (m)")

In [ ]:
# rotate the coordinates of the monitor well trajectory and the horizon
theta = -0.648114658
phi = 54.4

xyz_well=np.transpose(np.array([x,y,z]))
xyz_well_rot=np.copy(xyz_well)
for i in range(xyz_well.shape[0]):
    xyz_well_rot[i] = rotateYZ(xyz_well[i],theta,phi)

xyz_hrz=np.transpose(np.array([x_hrz,y_hrz,z_hrz]))
xyz_hrz_rot=np.copy(xyz_hrz)
for i in range(xyz_hrz.shape[0]):
    xyz_hrz_rot[i] = rotateYZ(xyz_hrz[i],theta,phi)

In [ ]:
# QC plot
plt.figure(figsize=(20,10))
plt.plot(xyz_hrz_rot[:,0],xyz_hrz_rot[:,2],linewidth=3,color='k',label="hrz")
plt.plot(xyz_well_rot[:,0],xyz_well_rot[:,2],linewidth=1,color='r',label="well")
plt.legend()
plt.grid()
plt.gca().invert_yaxis()
plt.gca().set_ylim(1955,1935)
plt.xlabel("X (m)")
plt.ylabel("Z (m)")

In [ ]:
# interpolate the horizon to a regular grid
n1=3689
o1=166
d1=0.5
hrz_dense=np.zeros((n1))
hrz_dense[0]=xyz_hrz_rot[0,2]
hrz_dense[n1-1]=xyz_hrz_rot[n_hrz-1,2]
for i in range(1,n1-1,1):
    j=0
    x1 = o1+i*d1
    while (x1 > xyz_hrz_rot[j,0] and j<xyz_hrz_rot.shape[0]-1):
        j+=1
    a = (x1-xyz_hrz_rot[j-1,0])/(xyz_hrz_rot[j,0] - xyz_hrz_rot[j-1,0])
    hrz_dense[i] = a*xyz_hrz_rot[j,2] + (1-a)*xyz_hrz_rot[j-1,2]

In [ ]:
# QC plot
ax = plt.axes(projection='3d')
ax.plot3D(xyz_hrz_rot[:,0],xyz_hrz_rot[:,1],xyz_hrz_rot[:,2],linewidth=3,color='k',label="coarse")
ax.plot3D(np.linspace(o1,o1+(n1-1)*d1,n1),np.zeros((n1))+np.median(xyz_hrz_rot[:,1]),hrz_dense[:],linewidth=1,color='r',label="dense")
plt.legend()
plt.gca().invert_zaxis()
plt.gca().set_ylim(-300,300)
plt.gca().set_zlim(1980,1940)
ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")
ax.set_zlabel("Z (m)")

In [ ]:
# save to disk and convert to seplib
outputpath='../dat/'

np.save(outputpath+'ch0_formation_bottom_hrz_rotated',hrz_dense)

!sep2numpy.py --input ../dat/ch0_formation_bottom_hrz_rotated.npy --output ../dat/ch0_formation_bottom_hrz_rotated.H --mode 1 --datapath ${PWD}/../dat/

!echo n1=3689 o1=166 d1=0.5 label1="X (m)" >> ../dat/ch0_formation_bottom_hrz_rotated.H

!rm -f ../dat/ch0_formation_bottom_hrz_rotated.npy